In [29]:
import sys
from pathlib import Path
# Add src directory to path to import organoid_analysis package
sys.path.insert(0, str(Path('..') / 'src'))
import napari
import pandas as pd
import numpy as np
from cellpose import models, core, io
import pyclesperanto_prototype as cle
from tifffile import imread 
from skimage.measure import regionprops_table
from organoid_analysis.utils import list_images, read_image, extract_scaling_metadata, remap_labels, segment_organoids_from_cp_labels, extract_organoid_stats_and_merge

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose models
model = models.CellposeModel(gpu=True, model_type="cyto3") 

creating new log file
2026-02-18 13:15:28,778 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2026-02-18 13:15:28,778 [INFO] 
cellpose version: 	3.1.1.3 
platform:       	win32 
python version: 	3.10.19 
torch version:  	2.5.0
2026-02-18 13:15:28,778 [INFO] ** TORCH CUDA version installed and working. **
2026-02-18 13:15:28,778 [INFO] >> cyto3 << model set to be used
2026-02-18 13:15:28,778 [INFO] ** TORCH CUDA version installed and working. **
2026-02-18 13:15:28,778 [INFO] >>>> using GPU (CUDA)
2026-02-18 13:15:28,929 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cyto3
2026-02-18 13:15:29,173 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [30]:
# Define the markers you wish to analyze and its cellular compartment (i.e. cell,membrane or nuclei)
# ("channel_name", position, location)
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane"), ("CellMask_AF647", 3, "membrane"), ("Occludin_RFP", 0, "cell"), ("Claudin_FITC", 1, "cell"), ("CellMask_AF647", 3, "cell"), ("DAPI", 2, "nuclei")]

In [31]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\P

In [32]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id from filename
well_id = filename.split("_")[0]

# Extract x,y,z scaling from .nd2 file metadata in order feed the Z pixel size / XY pixel size ratio into Cellpose
pixel_size_x, pixel_size_y, voxel_size_z = extract_scaling_metadata(image)

assert pixel_size_x == pixel_size_y
z_to_xy_ratio = voxel_size_z / pixel_size_x
print(z_to_xy_ratio)


Image analyzed: B2
Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm
3.017451031446116


In [33]:
# Type down the multiposition index you want to explore
position = 0

# (suffix, subdir, channels, cellprob_threshold) for each compartment
# channels: [cyto_chan, nuclear_chan] for cell; [nuclear_chan, grayscale] for nuclei (indices in cellpose_input 0=mem, 1=nuclei, 2=Cellmask)
segment_configs = [
    ("cell", "cellpose_cell_labels", [3, 2]),   # Cellmask (2), nuclei (1) in cellpose_input
    ("nuclei", "cellpose_nuclei_labels", [2, 0]),  # nuclei (1), grayscale
]

print(f"Analyzing multiposition index {position}")

# Open one of the multipositions in the img file
single_img = img[position]
# Input shape for next step has to be C, Z, Y, X
single_img = single_img.transpose(1, 0, 2, 3)

# Keep only 3 meaningful input fluorescence channels to mimick Cellpose GUI normalization
# 0 membrane marker, 2 nuclei, 3 Cellmask (Numpy 0-index) → cellpose_input: 0=mem, 1=nuclei, 2=Cellmask
cellpose_input = single_img[[0, 2, 3], :, :, :]

labels = {}
for suffix, subdir, channels in segment_configs:
    cellpose_filename = f"{well_id}_{position}_{suffix}"
    prediction_path = directory_path / subdir / f"{cellpose_filename}.tif"

    if prediction_path.exists():
        print(f"Predictions already calculated for: {cellpose_filename} ...loading")
        labels[suffix] = imread(prediction_path)
    else:
        labels[suffix], _, _ = model.eval(
            cellpose_input,
            channels=channels,
            diameter=20,
            do_3D=True,
            anisotropy=z_to_xy_ratio,
            normalize=True,
            flow_threshold=0.4,
            cellprob_threshold=0.0,
            min_size=15,
        )

del cellpose_input

cytoplasm_labels = labels["cell"]
nuclei_labels = labels["nuclei"]

# Remap nuclei labels to match their surrounding cytoplasm id
nuclei_labels = remap_labels(nuclei_labels, cytoplasm_labels)

# Initialize Napari Viewer
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(single_img)
viewer.add_labels(cytoplasm_labels)
viewer.add_labels(nuclei_labels)

Analyzing multiposition index 0
Predictions already calculated for: B2_0_cell ...loading
Predictions already calculated for: B2_0_nuclei ...loading


<Labels layer 'nuclei_labels' at 0x236b9298250>

In [34]:
# Create a dictionary containing all image descriptors
#TODO: Add multiposition index during BP
descriptor_dict = {
            "filename": filename,
            "well_id": well_id,
            }

props_list = []

# Single list of regionprops to request; rename_map is built from this below (all 3D-compatible)
regionprops_properties = [
    "label",
    "area",                          # number of voxels (volume in voxel units)
    "area_bbox",                     # volume of axis-aligned bounding box
    "area_filled",                   # volume after filling holes
    "axis_major_length",             # length of major axis from inertia tensor (elongation)
    "axis_minor_length",             # length of minor axis (second principal axis in 3D)
    "equivalent_diameter_area",      # diameter of sphere with same volume as region
    "euler_number",                  # topology: objects + holes − tunnels (connectivity)
    "extent",                        # volume / bounding-box volume (fill of the box)
    "inertia_tensor_eigvals",        # eigenvalues of inertia tensor (3 values: shape/orientation)
    "intensity_mean",
    "intensity_min",
    "intensity_max",
    "intensity_std",
]

membrane_labels = None # Variable to check in order to compute membrane_labels just once, avoid repeated GPU ops
locations_seen = set()  # track locations so we only keep shape columns once per location (avoids duplicate columns on merge)

# Loop through markers and extract 
for marker_name, ch_nr, location in markers:
    print(f"Analyzing channel: {marker_name} in {location} ...")

    # Region cell, nuclei or membrane: generate membrane and extract info from that location if needed.
    # Pick label image and optionally create membrane layer.
    if location == "cell":
        label_image = cytoplasm_labels
    elif location == "nuclei":
        label_image = nuclei_labels
    elif location == "membrane":
        layer_name = "membrane_labels"
        # Check if layer already exists in the viewer
        if layer_name not in viewer.layers:
            # Generate membrane if needed
            if membrane_labels is None:
                membrane_labels = cle.reduce_labels_to_label_edges(cytoplasm_labels)
                membrane_labels = cle.pull(membrane_labels)
            viewer.add_labels(membrane_labels, name=layer_name)
            label_image = membrane_labels
        else:
            # Layer already present → use its data
            label_image = viewer.layers[layer_name].data
            if membrane_labels is None:
                membrane_labels = label_image
    else:
        raise ValueError(f"Unknown location: {location}")

    props = regionprops_table(
        label_image=label_image,
        intensity_image=single_img[ch_nr],
        properties=regionprops_properties,
    )

    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename columns to indicate the specific image; build rename_map from regionprops_properties:
    # label unchanged; area -> location_area; intensity_* -> prefix_*_int (concentration proxy etc.)
    prefix = f"{location}_{marker_name}"
    rename_map = {"label": "label"}
    for prop in regionprops_properties:
        if prop == "label":
            continue
        elif prop == "area":
            rename_map[prop] = f"{location}_area"
        elif prop.startswith("intensity_"):
            suffix = prop.replace("intensity_", "")
            rename_map[prop] = f"{prefix}_{suffix}_int"
        else:
            rename_map[prop] = f"{location}_{prop}"

    props_df.rename(columns=rename_map, inplace=True)

    # Derived columns (use names from rename_map so they stay in sync with regionprops_properties)
    mean_col = rename_map["intensity_mean"]
    max_col = rename_map["intensity_max"]
    area_col = rename_map["area"]
    # Max / mean ratio (puncta vs diffuse signal)
    props_df[f"{prefix}_max_mean_ratio"] = props_df[max_col] / props_df[mean_col].replace(0, np.nan)
    # Total marker content per cell
    props_df[f"{prefix}_sum_int"] = props_df[mean_col] * props_df[area_col]

    # If we already have shape columns for this location (e.g. second membrane marker), keep only label and marker-specific columns to avoid MergeError on duplicate column names
    if location in locations_seen:
        cols_to_keep = ["label"] + [c for c in props_df.columns if c.startswith(prefix + "_")]
        props_df = props_df[cols_to_keep]
    else:
        locations_seen.add(location)

    # Append each props_df to props_list
    props_list.append(props_df)

# Initialize the df with the first df in the list
props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0    
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

# Obtain rough single organoid outlines by fusing Cellpose labels and dilation, merging, erosion morphological operations
mip_labels, organoid_labels = segment_organoids_from_cp_labels(cytoplasm_labels)
viewer.add_labels(organoid_labels)

# Display final stats (including per cell and per organoid)
final_df = extract_organoid_stats_and_merge(mip_labels, organoid_labels, props_df)
final_df

Analyzing channel: Occludin_RFP in membrane ...
Analyzing channel: Claudin_FITC in membrane ...
Analyzing channel: CellMask_AF647 in membrane ...
Analyzing channel: Occludin_RFP in cell ...
Analyzing channel: Claudin_FITC in cell ...
Analyzing channel: CellMask_AF647 in cell ...
Analyzing channel: DAPI in nuclei ...
Cells mapped to no organoid: 191 - 12.28% of total cells (1556)


,filename,well_id,organoid,label,membrane_area,membrane_area_bbox,membrane_area_filled,membrane_axis_major_length,membrane_axis_minor_length,membrane_equivalent_diameter_area,...,nuclei_DAPI_min_int,nuclei_DAPI_max_int,nuclei_DAPI_std_int,nuclei_DAPI_max_mean_ratio,nuclei_DAPI_sum_int,organoid_area,organoid_perimeter,organoid_eccentricity,organoid_solidity,organoid_extent
0,B2,B2,0,1,211.0,880.0,211.0,13.635739,9.481485,7.386317,...,366.0,1390.0,213.271758,2.016370,94442.0,NaN,NaN,NaN,NaN,NaN
1,B2,B2,0,2,485.0,2926.0,485.0,22.957258,12.713338,9.747974,...,351.0,3756.0,680.632604,2.978715,707391.0,NaN,NaN,NaN,NaN,NaN
2,B2,B2,1,3,580.0,3179.0,580.0,21.764693,13.380577,10.346889,...,374.0,2071.0,382.631708,2.195016,514208.0,6643.0,326.007143,0.462363,0.951174,0.793668
3,B2,B2,0,4,555.0,3179.0,555.0,22.149119,13.580842,10.196038,...,289.0,2112.0,419.890176,2.112856,814670.0,NaN,NaN,NaN,NaN,NaN
4,B2,B2,0,5,205.0,729.0,205.0,12.019814,10.361562,7.315630,...,443.0,4095.0,906.785594,2.837823,382397.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,B2,B2,18,1551,72.0,182.0,72.0,13.784459,2.190823,5.161524,...,454.0,922.0,90.668802,1.437561,89791.0,35073.0,813.369624,0.680631,0.938559,0.734129
1320,B2,B2,0,1552,144.0,600.0,144.0,22.128512,1.774106,6.503113,...,301.0,879.0,133.392057,1.651326,287974.0,NaN,NaN,NaN,NaN,NaN
1321,B2,B2,0,1553,72.0,200.0,72.0,12.352164,1.524680,5.161524,...,324.0,738.0,89.036945,1.495715,133714.0,NaN,NaN,NaN,NaN,NaN
1322,B2,B2,0,1554,44.0,96.0,44.0,10.014523,2.192167,4.380107,...,294.0,960.0,134.574847,1.878022,179934.0,NaN,NaN,NaN,NaN,NaN
